In [3]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [4]:
import pandas as pd
import numpy as np
import sys, os
import h5py
import logging
from datetime import datetime

In [5]:
import h5py
import numpy as np
import os, sys
import subprocess
import glob
from statsmodels import robust

In [6]:
pip show ont_fast5_api #make sure the ont_fast5_api is the latest version

Name: ont-fast5-api
Version: 4.0.0
Summary: Oxford Nanopore Technologies fast5 API software
Home-page: https://github.com/nanoporetech/ont_fast5_api
Author: Oxford Nanopore Technologies, Limited
Author-email: 
License: MPL 2.0
Location: /pod/2/li-lab/Ziwei/Anaconda3/envs/nanomodel/lib/python3.6/site-packages
Requires: progressbar33, numpy, packaging, h5py
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [7]:
sys.version

'3.6.13 | packaged by conda-forge | (default, Sep 23 2021, 07:56:31) \n[GCC 9.4.0]'

# Pre-precess the data

The fast5 file is copied from /fastscratch/c-panz/2021-09-30/guppy5.0.14_supmodel_tombo/T4LambdaTF1/0/workspace

input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/

00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5

000006ea-dddb-429c-8277-a81ce50da6a0.fast5

The resquiggled signal is saved under:

correctedGroup="RawGenomeCorrected_001"

basecallGroup="Basecall_1D_001"

## Open multiple files

In [8]:
#Import all files under a folder into a list
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/'
from ont_fast5_api.conversion_tools.conversion_utils import get_fast5_file_list

fast5_fp = get_fast5_file_list(input_path,recursive=True)
logging.info("Found {} files to process.".format(len(fast5_fp)))

In [9]:
fast5_fp

['/pod/2/li-lab/Ziwei/Nanopore/daily/test/000006ea-dddb-429c-8277-a81ce50da6a0.fast5',
 '/pod/2/li-lab/Ziwei/Nanopore/daily/test/00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5']

In [20]:
for fast5_fn in fast5_fp:
#    print(fast5_fn)
    with h5py.File(fast5_fn, mode="r") as f5:
        print(f5)

<HDF5 file "000006ea-dddb-429c-8277-a81ce50da6a0.fast5" (mode r)>
<HDF5 file "00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5" (mode r)>


## Single fast5 file

In [21]:
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/'
fast5_fn='00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5'

fast5_fn=os.path.join(input_path, fast5_fn)
f5 = h5py.File(fast5_fn, mode="r") 
f5

<HDF5 file "00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5" (mode r)>

In [34]:
signal_group='Raw/Reads'
corr_group='RawGenomeCorrected_001' #tombo resquiggle save location
basecall_group='Basecall_1D_001' #has to be save in the tombo requiggle step
basecall_subgroup='BaseCalled_template' #Attention: the basecall_subgroup can be 'basecalled_compl

## Basecall info

In [23]:
basecall = f5[f'/Analyses/{corr_group}/{basecall_subgroup}']
basecall_attr = dict(list(basecall.attrs.items()))
basecall_attr

{'lower_lim': -5.0,
 'norm_type': 'median',
 'outlier_threshold': 5.0,
 'rna': False,
 'scale': 52.47260793234401,
 'shift': 453.68545996959926,
 'signal_match_score': 0.7739355451096712,
 'status': 'success',
 'upper_lim': 5.0}

### Get event information

In [24]:
# Get event data after tombo-requirrel
event = f5[f'/Analyses/{corr_group}/{basecall_subgroup}/Events']
corr_attr = dict(list(event.attrs.items()))
corr_attr

{'read_start_rel_to_raw': 1450}

In [25]:
read_start_rel_to_raw = corr_attr['read_start_rel_to_raw']
print('read_start_rel_to_raw: {}'.format(read_start_rel_to_raw))

read_start_rel_to_raw: 1450


In [26]:
#Check the event column attri
event.dtype, event.dtype.names

(dtype([('norm_mean', '<f8'), ('norm_stdev', '<f8'), ('start', '<u4'), ('length', '<u4'), ('base', 'S1')]),
 ('norm_mean', 'norm_stdev', 'start', 'length', 'base'))

In [27]:
event['length'][1:10]

array([ 4,  3, 14,  7,  5,  3,  5,  3,  3], dtype=uint32)

In [18]:
sum(event['length'])

293395

### Event normalization

### Extract event normalization

In [28]:
#Calculate the start position relatively to the reference genome
start = list(map(lambda x: x + read_start_rel_to_raw, event['start']))
#Get the event length for each base
length = event['length'].astype(np.int)
#Get the base information
base = [x.decode("UTF-8") for x in event['base']]

assert len(start) == len(base) == len(length)
events = list(zip(start, length, base))
assert len(events) == len(start) == len(base) == len(length)

#Save the start position, event length, base info
events[0:10]

[(1450, 5, 'A'),
 (1455, 4, 'T'),
 (1459, 3, 'T'),
 (1462, 14, 'T'),
 (1476, 7, 'C'),
 (1483, 5, 'T'),
 (1488, 3, 'G'),
 (1491, 5, 'A'),
 (1496, 3, 'A'),
 (1499, 3, 'G')]

In [29]:
type(events)

list

### Get strand alignment information

In [100]:
signal_group='Raw/Reads'
corr_group='RawGenomeCorrected_001' #tombo resquiggle save location
basecall_group='Basecall_1D_001' #has to be save in the tombo requiggle step
basecall_subgroup='BaseCalled_template' #Attention: the basecall_subgroup can be 'basecalled_compl

input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/000006ea-dddb-429c-8277-a81ce50da6a0.fast5'

f5 = h5py.File(input_path, mode="r")

strand_alignment = f5[f'/Analyses/{corr_group}/{basecall_subgroup}/Alignment']
alignment_attr = dict(list(strand_alignment.attrs.items()))
alignment_attr

{'clipped_bases_end': 85,
 'clipped_bases_start': 0,
 'mapped_chrom': 'chr11',
 'mapped_end': 77052439,
 'mapped_start': 77036077,
 'mapped_strand': '-',
 'num_deletions': 140,
 'num_insertions': 66,
 'num_matches': 16143,
 'num_mismatches': 79}

In [31]:
#Extract chrom, chrom_start, strand for alignment information
chrom = alignment_attr['mapped_chrom']
strand = alignment_attr['mapped_strand']
chrom_start = alignment_attr['mapped_start']
chrom, strand, chrom_start

('chr4', '-', 118511944)

In [32]:
#Assign strand information
strand = 't' if basecall_subgroup=='BaseCalled_template' else 'c'
strand

't'

## Get raw signal & read_id

In [62]:
read_num = list(f5[f'/{signal_group}'])[0]
signal = f5[f'/{read_group}/{read_num}/Signal']
signal_attr = np.array(signal[:])

read = f5[f'/{read_group}/{read_num}']
read_attr = dict(list(read.attrs.items()))
read_id = read_attr['read_id']

signal_attr, read_id

(array([434, 432, 465, ..., 359, 380, 777], dtype=int16),
 b'00000156-e575-4fb7-9053-d00dbe5c8d9c')

## signal rescalling

In [65]:
#https://github.com/grimbough/IONiseR/blob/47d8ab1e1d798f3591407be679076a1a5b5d9dd2/R/squiggle.R#L81

#channels 0pA adc: /UniqueGlobalKey/channel_id/{offset}
#digitisable range in pA: /UniqueGlobalKey/channel_id/{range}
#digitisation: /UniqueGlobalKey/channel_id/{digitisation}
#current in pA = (signal_value + channels 0pA adc) * digitisable range in pA / digitisation
import os
channel_group='UniqueGlobalKey/channel_id'

fast5_fn = os.path.join(input_path, fast5_fn)
f5 = h5py.File(fast5_fn, mode="r")

channel = f5[f'/{channel_group}']
channel_attr = dict(list(channel.attrs.items()))
current = np.array((signal_attr + channel_attr['offset']) * channel_attr['range'] / channel_attr['digitisation'])
current

array([ 78.12429607,  77.76918563,  83.62850784, ...,  64.80765469,
        68.53631428, 139.02573596])

### signal normorlization

In [66]:
from statsmodels import robust
method = 'mad'
signal_new = current

if method == 'mad':
#assuming a normal distribution 
#https://stackoverflow.com/a/40619633
    shift, scale = np.median(signal_new), np.float(robust.mad(signal_new))
elif method == 'zscore':
    shift, scale = np.mean(signal_new), np.float(np.std(signal_new))
else:
    raise ValueError('Normalized method not recogized')
            
#signal normalization: https://nanoporetech.github.io/tombo/resquiggle.html#tombo-fast5-format
# There may be problem after tombo 1.3, see explanation above
norm_signal = (signal - shift) / scale #POTENTIAL ISSUE!
assert len(signal) == len(norm_signal)
norm_signal = np.around(norm_signal, 6)
norm_signal

array([24.383541, 24.245404, 26.52466 , ..., 19.203413, 20.653848,
       48.073992])

# Define fast5 class with function features

In [103]:
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/'
#    fast5_fn='00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5'
from ont_fast5_api.conversion_tools.conversion_utils import get_fast5_file_list
fast5_fp = get_fast5_file_list(input_path,recursive=True)
for fast5_fn in fast5_fp:
    print(fast5_fn)

/pod/2/li-lab/Ziwei/Nanopore/daily/test/000006ea-dddb-429c-8277-a81ce50da6a0.fast5
/pod/2/li-lab/Ziwei/Nanopore/daily/test/00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5


In [92]:
signal_group='Raw/Reads'
corr_group='RawGenomeCorrected_001' #tombo resquiggle save location
basecall_group='Basecall_1D_001' #has to be save in the tombo requiggle step
basecall_subgroup='BaseCalled_template' #Attention: the basecall_subgroup can be 'basecalled_compl

class raw_fast5:
    def __init__(self, path, corr_group, basecall_group, basecall_subgroup, signal_group):
        self._fast5_id = os.path.basename(path)
        self._obj = h5py.File(path, mode="r")
        self._corr_group = corr_group
        self._basecall_group = basecall_group
        self._basecall_subgroup = basecall_subgroup
        self._signal_group = signal_group
        
    def __eq__(self, other):
        self._fast5_id == other._fast5_id
        
    def fast5_readid(self):
        f5 = self._obj
        try:
            read_num = list(f5[f'/{signal_group}'])[0]
        except Exception:
            raise RuntimeError('Signal not found.')
                
        read = f5[f'/{read_group}/{read_num}']
        read_attr = dict(list(read.attrs.items()))
        read_id = read_attr['read_id'].decode('UTF-8') 
        return read_id
    
    def fast5_signal(self):
        f5 = self._obj
        try:
            read_num = list(f5[f'/{signal_group}'])[0]
        except Exception:
            raise RuntimeError('Signal not found.')
            
        signal = f5[f'/{read_group}/{read_num}/Signal']
        signal_attr = np.array(signal[:])
        return signal_attr
    
    def fast5_event(self):
        f5 = self._obj
        try:
            # Get event data after tombo-requirrel
            event = f5[f'/Analyses/{self._corr_group}/{self._basecall_subgroup}/Events']
        except Exception:
            raise RuntimeError('Events not found.')
            
        corr_attr = dict(list(event.attrs.items()))
        
        #get read location relatively to the reference genome
        read_start_rel_to_raw = corr_attr['read_start_rel_to_raw']   
        
        #Calculate the start position relatively to the reference genome
        start = list(map(lambda x: x + read_start_rel_to_raw, event['start']))
        #Get the event length for each base
        length = event['length'].astype(np.int)
        #Get the seq information
        seq = [x.decode("UTF-8") for x in event['base']]
        
        assert len(events) == len(start) == len(seq) == len(length)
        event_attr = list(zip(start, length, base))
        return event_attr
    
    def fast5_align(self):
        f5 = self._obj
        try:
            strand_align = f5[f'/Analyses/{self._corr_group}/{self._basecall_subgroup}/Alignment']
        except Exception:
            raise RuntimeError('Alignment not found.')
            
        read_strand = 't' if self._basecall_subgroup=='BaseCalled_template' else 'c'
        
        align_attr = dict(list(strand_align.attrs.items()))
        #Extract chrom, chrom_start, strand for alignment information
        chrom = align_attr['mapped_chrom']
        strand = align_attr['mapped_strand']
        start = align_attr['mapped_start']
        
        return chrom, strand, start, read_strand

### Define signal rescalling

In [93]:
# Signal rescalling
#https://github.com/grimbough/IONiseR/blob/47d8ab1e1d798f3591407be679076a1a5b5d9dd2/R/squiggle.R#L81
#channels 0pA adc: /UniqueGlobalKey/channel_id/{offset}
#digitisable range in pA: /UniqueGlobalKey/channel_id/{range}
#digitisation: /UniqueGlobalKey/channel_id/{digitisation}
#current in pA = (signal_value + channels 0pA adc) * digitisable range in pA / digitisation
import os
def rescale_fast5_signal(input_path, signal_attr,
                         channel_group='UniqueGlobalKey/channel_id'):
    f5 = h5py.File(input_path, mode="r")
    channel = f5[f'/{channel_group}']
    channel_attr = dict(list(channel.attrs.items()))
    current = np.array((signal_attr + channel_attr['offset']) * channel_attr['range'] / channel_attr['digitisation'])
    return current

### Define signal normalization

In [94]:
# Signal normalization with mad method
def fast5_normalized_signal(signal, method='mad'):
    if method == 'mad':
        #assuming a normal distribution 
        #https://stackoverflow.com/a/40619633
        shift, scale = np.median(signal), np.float(statsmodels.robust.mad(signal))
    elif method == 'zscore':
        shift, scale = np.mean(signal), np.float(np.std(signal))
    else:
        raise ValueError('Normalized method not recogized')
            
    #signal normalization: https://nanoporetech.github.io/tombo/resquiggle.html#tombo-fast5-format
    # There may be problem after tombo 1.3, see explanation above
    norm_signal = (signal - shift) / scale #POTENTIAL ISSUE!
    assert len(signal) == len(norm_signal)
    norm_signal = np.around(norm_signal, 6)
    return norm_signal

# Test

In [95]:
import os
signal_group='Raw/Reads'
corr_group='RawGenomeCorrected_001' #tombo resquiggle save location
basecall_group='Basecall_1D_001' #has to be save in the tombo requiggle step
basecall_subgroup='BaseCalled_template' #Attention: the basecall_subgroup can be 'basecalled_compl

input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/'
fast5_fn='00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5'
fast5_fn= os.path.join(input_path, fast5_fn)

In [97]:
f5 = raw_fast5(fast5_fn, corr_group, basecall_group, basecall_subgroup, signal_group)
event = f5.fast5_event()
read_id = f5.fast5_readid()
signal = f5.fast5_signal()
chrom, strand, start, read_strand = f5.fast5_align()
chrom, strand, start, read_strand

('chr4', '-', 118511944, 't')

In [90]:
event[0:5]

[(1450, 5, 'A'),
 (1455, 4, 'T'),
 (1459, 3, 'T'),
 (1462, 14, 'T'),
 (1476, 7, 'C')]

In [84]:
current_rescaled = rescale_fast5_signal(fast5_fn, signal)
current_rescaled

array([ 78.12429607,  77.76918563,  83.62850784, ...,  64.80765469,
        68.53631428, 139.02573596])

In [86]:
norm_signal = fast5_normalized_signal(current_rescaled)
norm_signal

array([-0.196215, -0.220742,  0.183952, ..., -1.115974, -0.858442,
        4.010148])